##  CNN做MNIST分类
refer: https://github.com/hjptriplebee/AlexNet_with_tensorflow/blob/master/alexnet.py

In [1]:
import numpy as np
import tensorflow as tf

# 设置按需使用GPU
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 1.导入数据，用 tensorflow 导入

In [3]:
# 用tensorflow 导入数据
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
# 看看咱们样本的数量
print(mnist.test.labels.shape)
print(mnist.train.labels.shape)

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions f

## 2. 构建网络

In [4]:
# 定义卷积层
def conv2d(x, filter_shape, strides_x, strides_y, padding, name):
    """ 
    Args:
        x: 4-D inputs. [batch_size, in_height, in_width, in_channels]
        filter_shape: A list of ints.[filter_height, filter_width, in_channels, out_channels]
        strides: A list of ints. 1-D tensor of length 4. The stride of the sliding window for each dimension of input.
        padding: A string from: "SAME", "VALID". The type of padding algorithm to use.
    Returns:
        h_conv:  A 4-D tensor. [batch_size, out_height, out_width, out_channels]. 
        if padding is 'SAME', then out_height==in_height. 
        else, out_height = in_height - filter_height + 1.
        the same for out_width.
    """
    assert padding in ['SAME', 'VALID']
    strides=[1,strides_x, strides_y,1]
    with tf.variable_scope(name):
        W_conv = tf.get_variable('w', shape=filter_shape)
        b_conv = tf.get_variable('b', shape=[filter_shape[-1]])
        h_conv = tf.nn.conv2d(x, W_conv, strides=strides, padding=padding)
        h_conv_relu = tf.nn.relu(h_conv + b_conv)
    return h_conv_relu
    

def max_pooling(x, k_height, k_width, strides_x, strides_y, padding='SAME'):
    """max pooling layer."""
    ksize=[1,k_height, k_width,1]
    strides=[1,strides_x, strides_y,1]
    h_pool = tf.nn.max_pool(x, ksize, strides, padding)
    return h_pool

def dropout(x, keep_prob, name=None):
    """dropout layer"""
    return tf.nn.dropout(x, keep_prob, name=name)

def fc(x, in_size, out_size, name, activation=None):
    """fully-connect
    Args:
        x: 2-D tensor, [batch_size, in_size]
        in_size: the size of input tensor.
        out_size: the size of output tensor.
        activation: 'relu' or 'sigmoid' or 'tanh'.
    Returns:
        h_fc: 2-D tensor, [batch_size, out_size].
    """
    if activation is not None:
        assert activation in ['relu', 'sigmoid', 'tanh'], 'Wrong activation function.'
    with tf.variable_scope(name):
        w = tf.get_variable('w', shape = [in_size, out_size], dtype=tf.float32)
        b = tf.get_variable('b', [out_size], dtype=tf.float32)
        h_fc = tf.nn.xw_plus_b(x, w, b)
        if activation == 'relu':
            return tf.nn.relu(h_fc)
        elif activation == 'tanh':
            return tf.nn.tanh(h_fc)
        elif activation == 'sigmoid':
            return tf.nn.sigmoid(h_fc)
        else:
            return h_fc

with tf.name_scope('inputs'):
    X_ = tf.placeholder(tf.float32, [None, 784])
    y_ = tf.placeholder(tf.float32, [None, 10])

# 把X转为卷积所需要的形式
X = tf.reshape(X_, [-1, 28, 28, 1])
h_conv1 = conv2d(X, [5, 5, 1, 32], 1, 1, 'SAME', 'conv1')
h_pool1 = max_pooling(h_conv1, 2, 2, 2, 2)

h_conv2 = conv2d(h_pool1, [5, 5, 32, 64], 1, 1, 'SAME', 'conv2')
h_pool2 = max_pooling(h_conv2, 2, 2, 2, 2)

# flatten
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = fc(h_pool2_flat, 7*7*64, 1024, 'fc1', 'relu')

# dropout: 输出的维度和h_fc1一样，只是随机部分值被值为零
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
h_fc2 = fc(h_fc1_drop, 1024, 10, 'fc2')
y_conv = tf.nn.softmax(h_fc2)
print('Finished building network.')

Finished building network.


## 3.训练和评估

<b> 在测试的时候不使用 mini_batch， 那么测试的时候会占用较多的GPU（4497M），这在 notebook 交互式编程中是不推荐的。

In [5]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.global_variables_initializer())
for i in range(10000):
    batch = mnist.train.next_batch(50)
    if i%1000 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            X_:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={X_: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    X_: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.06
step 1000, training accuracy 0.94
step 2000, training accuracy 1
step 3000, training accuracy 1
step 4000, training accuracy 1
step 5000, training accuracy 1
step 6000, training accuracy 1
step 7000, training accuracy 1
step 8000, training accuracy 0.98
step 9000, training accuracy 1
test accuracy 0.9922


<b> 下面改成了 test 也用 mini_batch 的形式， 显存只用了 529M,所以还是很成功的。

In [6]:
# 题外话：在做这个例子的过程中遇到过：资源耗尽的错误，为什么？
# -> 因为之前每次做 train_acc  的时候用了全部的 55000 张图，显存爆了.

# 1.损失函数：cross_entropy
cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))
# 2.优化函数：AdamOptimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# 3.预测准确结果统计
#　预测值中最大值（１）即分类结果，是否等于原始标签中的（１）的位置。argmax()取最大值所在的下标
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.arg_max(y_, 1))  
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


# 如果一次性来做测试的话，可能占用的显存会比较多，所以测试的时候也可以设置较小的batch来看准确率
test_acc_sum = tf.Variable(0.0)
batch_acc = tf.placeholder(tf.float32)
new_test_acc_sum = tf.add(test_acc_sum, batch_acc)
update = tf.assign(test_acc_sum, new_test_acc_sum)

# 定义了变量必须要初始化，或者下面形式
sess.run(tf.global_variables_initializer())
# 或者某个变量单独初始化 如：
# x.initializer.run()

# 训练
for i in range(5000):
    X_batch, y_batch = mnist.train.next_batch(batch_size=50)
    if i % 500 == 0:
        train_accuracy = accuracy.eval(feed_dict={X_: X_batch, y_: y_batch, keep_prob: 1.0})
        print("step %d, training acc %g" % (i, train_accuracy))
    train_step.run(feed_dict={X_: X_batch, y_: y_batch, keep_prob: 0.5})  

# 全部训练完了再做测试，batch_size=100
for i in range(100): 
    X_batch, y_batch = mnist.test.next_batch(batch_size=100)
    test_acc = accuracy.eval(feed_dict={X_: X_batch, y_: y_batch, keep_prob: 1.0})
    update.eval(feed_dict={batch_acc: test_acc})
    if (i+1) % 20 == 0:
        print("testing step %d, test_acc_sum %g" % (i+1, test_acc_sum.eval()))
print(" test_accuracy %g" % (test_acc_sum.eval() / 100.0))

Instructions for updating:
Use `argmax` instead
step 0, training acc 0.1
step 500, training acc 0.96
step 1000, training acc 0.86
step 1500, training acc 0.98
step 2000, training acc 0.98
step 2500, training acc 0.98
step 3000, training acc 0.98
step 3500, training acc 1
step 4000, training acc 1
step 4500, training acc 0.98
testing step 20, test_acc_sum 19.86
testing step 40, test_acc_sum 39.58
testing step 60, test_acc_sum 59.34
testing step 80, test_acc_sum 79.08
testing step 100, test_acc_sum 98.89
 test_accuracy 0.9889
